In [9]:
import pkg_resources
for d in pkg_resources.working_set:
     if str(d)[0:3] == "ff3" or str(d)[0:12] == "pycryptodome":
          print(str(d))

StatementMeta(testhmacspark, 46, 1, Finished, Available)

pycryptodome 3.17
ff3 1.0.1


In [43]:
%%pyspark
#df = spark.read.load.('abfss://bronze@testhmacmaskstor.dfs.core.windows.net/SalesLT.Customer.txt', format='text', header='true')
df = spark.read.option("header",True).csv('abfss://bronze@testhmacmaskstor.dfs.core.windows.net/SalesLT.Customer.txt')

display(df.limit(10))
#https://www.wheelodex.org/projects/ff3/wheels/ff3-1.0.1-py3-none-any.whl/

StatementMeta(testhmacspark, 46, 35, Finished, Available)

SynapseWidget(Synapse.DataFrame, aba9a6dd-51ea-4b19-b048-092ac4bc901d)

In [59]:
from ff3 import FF3Cipher
import string
#
KEY = TokenLibrary.getSecret('test-hmacmask-akv', 'fpekey', 'AzureKeyVaultHMAC')
TWEAK = TokenLibrary.getSecret('test-hmacmask-akv', 'fpetweak', 'AzureKeyVaultHMAC')
#
C_NUMERIC_MIN = 6
C_NUMERIC_MAX = 56
C_NUMERIC_RADIX = 10
C_NUMERIC_MIN_PREFIX = "0"
#
C_ALPHANUMERIC_MIN = 6
C_ALPHANUMERIC_MAX = 32
C_ALPHANUMERIC_RADIX = 62
C_ALPHANUMERIC_MIN_PREFIX = "0"
#
C_ALPHANUMERIC_EXTENDED_MIN = 4
C_ALPHANUMERIC_EXTENDED_MAX = 30
C_ALPHANUMERIC_EXTENDED_MIN_PREFIX = " "
C_ALPHANUMERIC_EXTENDED_ALPHABET = string.digits + string.ascii_lowercase + string.ascii_uppercase + " !@#$%^&*()?'\/-.,"
#
C_EMAIL_MIN = 4
C_EMAIL_MAX = 30
C_EMAIL_MIN_PREFIX = " "
C_EMAIL_ALPHABET = string.digits + string.ascii_lowercase + string.ascii_uppercase + "._%+-"

StatementMeta(testhmacspark, 46, 51, Finished, Available)

In [84]:
def _fpe_ff3_base(col_val, c, c_min, c_max, c_min_prefix):

    ciphertext =""
    if len(col_val) < c_min:
        # column length is too short to be encrypted. Prefix column with values to reach minimal length
        col_val = str(col_val).rjust(c_min, c_min_prefix)
        ciphertext = c.encrypt(col_val)
    elif len(col_val) > c_max:
        # column length is too long and needs to be chunked. Every chunck is then encrypted and appended
        current_chunck_length = c_max
        while current_chunck_length > 0:
            col_val_array = [col_val[i:i+current_chunck_length] for i in range(0, len(col_val), current_chunck_length)]
            # check if length of last chunck is larger than minimum
            if len(col_val_array[-1]) < c_min:
                #length of last element is shorter than minimum. Decrease current_chunck_length with 1
                current_chunck_length -= 1
            else:
                # length of last chunck is larger than minimum. Ready process
                break
        
        for col_val_item in col_val_array:
            ciphertext += c.encrypt(col_val_item)
    else:
        ciphertext = c.encrypt(col_val)

    return ciphertext

def _fpe_ff3_phone_base(col_val, c):

    raw_string = str(''.join(filter(str.isdigit, col_val)))
    raw_ciphertext = c.encrypt(raw_string)

    ciphertext = ""
    non_numeric = 0
    for j, val in enumerate(col_val):
        if not(val.isnumeric()):
            ciphertext += col_val[j]
            non_numeric += 1
        else:
            ciphertext += raw_ciphertext[j-non_numeric]
    
    return ciphertext

def _fpe_ff3_email_base(col_val, c):

    at_split = col_val.split("@")
    len_at_split = len(at_split[0])
    dot_split = at_split[1].split(".")
    len_dot_split = len_at_split + len(dot_split[0])
    raw_string = at_split[0] + dot_split[0] + dot_split[1]
    raw_ciphertext = c.encrypt(raw_string)
    ciphertext = raw_ciphertext[0 : len_at_split] + "@" + \
                 raw_ciphertext[len_at_split : len_dot_split] + "." + \
                 col_val[len_dot_split + 2 : ]
    
    return ciphertext

def fpe_ff3_numeric(col_val):
    c = FF3Cipher(KEY, TWEAK, radix=C_NUMERIC_RADIX)
    return _fpe_ff3_base(col_val, c, C_NUMERIC_MIN, C_NUMERIC_MAX, C_NUMERIC_MIN_PREFIX)

def fpe_ff3_alphanumeric(col_val):
    c = FF3Cipher(KEY, TWEAK, radix=C_ALPHANUMERIC_RADIX)
    return _fpe_ff3_base(col_val, c, C_ALPHANUMERIC_MIN, C_ALPHANUMERIC_MAX, C_ALPHANUMERIC_MIN_PREFIX)

def fpe_ff3_alphanumeric_extended(col_val):
    c = FF3Cipher.withCustomAlphabet(KEY, TWEAK, alphabet=C_ALPHANUMERIC_EXTENDED_ALPHABET)
    return _fpe_ff3_base(col_val, c, C_ALPHANUMERIC_EXTENDED_MIN, C_ALPHANUMERIC_EXTENDED_MAX, C_ALPHANUMERIC_EXTENDED_MIN_PREFIX)

def fpe_ff3_phone(col_val):
    c = FF3Cipher(KEY, TWEAK, radix=C_NUMERIC_RADIX)
    return _fpe_ff3_phone_base(col_val, c)

def fpe_ff3_email(col_val):
    c = FF3Cipher.withCustomAlphabet(KEY, TWEAK, alphabet=C_EMAIL_ALPHABET)
    return _fpe_ff3_email_base(col_val, c)

id = "1"
id_fpe = fpe_ff3_numeric(id)
name = "Bremer"
name_fpe = fpe_ff3_alphanumeric(name)
company = "Bremer & Sons!, LTD."
company_fpe = fpe_ff3_alphanumeric_extended(company)
phone = "06-23112312"
phone_fpe = fpe_ff3_phone(phone)
email = "bremersons@hotmail.com"
email_fpe = fpe_ff3_email(email)
print(id, "=>",  id_fpe)
print(name, "=>",  name_fpe)
print(company, "=>",  company_fpe)
print(phone, "=>",  phone_fpe)
print(email, "=>",  email_fpe)

StatementMeta(testhmacspark, 46, 76, Finished, Available)

1 => 359228
Bremer => 6paCYa
Bremer & Sons!, LTD. => $-TcfV8Y(KvxThJWE-f4
06-23112312 => 48-78322271
bremersons@hotmail.com => kksT1%2ITp@gQerH8Y.com


In [78]:
# https://tunetotech.com/Post.aspx?post=9
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from ff3 import FF3Cipher
#
fpe_ff3_numeric_udf = udf(fpe_ff3_numeric, StringType())
fpe_ff3_alphanumeric_udf = udf(fpe_ff3_alphanumeric, StringType())
fpe_ff3_alphanumeric_extended_udf = udf(fpe_ff3_alphanumeric_extended, StringType())
fpe_ff3_email_udf = udf(fpe_ff3_email, StringType())
fpe_ff3_phone_udf = udf(fpe_ff3_phone, StringType())
#
df2=df.withColumn('CustomerID', fpe_ff3_numeric_udf(df.CustomerID)) \
       .withColumn('FirstName', fpe_ff3_alphanumeric_udf(df.FirstName)) \
       .withColumn('LastName', fpe_ff3_alphanumeric_extended_udf(df.LastName)) \
       .withColumn('CompanyName', fpe_ff3_alphanumeric_extended_udf(df.CompanyName)) \
       .withColumn('EmailAddress', fpe_ff3_email_udf(df.EmailAddress)) \
       .withColumn('Phone', fpe_ff3_phone_udf(df.Phone))
#
display(df2.limit(10))

StatementMeta(testhmacspark, 46, 70, Finished, Available)

SynapseWidget(Synapse.DataFrame, bc4fe73a-57a9-4ec2-b51e-3681d5fe785e)

In [70]:
col_val = "bremerov@hotmail.com"
at_split = col_val.split("@")
len_at_split = len(at_split[0])
dot_split = at_split[1].split(".")
len_dot_split = len(dot_split[0])
len_dot_end = len(dot_split[1])

print (col_val[len_at_split+1: len_dot_split])

#import re
#def check(s):
#    pat = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,7}\b'
#    if re.match(pat,s):
#        print("Valid Email")
#    else:
#        print("Invalid Email")

#email="brem+erov@hotmail-.com"
#check(email)

StatementMeta(testhmacspark, 46, 62, Finished, Available)